In [18]:
!pip install jupyter-black "rich[jupyter]"
%load_ext jupyter_black
%load_ext rich

The jupyter_black extension is already loaded. To reload it, use:
  %reload_ext jupyter_black


In [19]:
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    DataCollatorWithPadding,
    AutoModelForSequenceClassification,
)
from torch.utils.data import DataLoader

In [20]:
raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)


def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)


tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [21]:
tokenized_datasets = tokenized_datasets.remove_columns(
    ["sentence1", "sentence2", "idx"]
)
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format("torch")
tokenized_datasets


DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 408
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1725
    })
})

In [22]:
train_dataloader = DataLoader(
    tokenized_datasets["train"], shuffle=True, batch_size=8, collate_fn=data_collator
)
valid_dataloader = DataLoader(
    tokenized_datasets["validation"],
    shuffle=True,
    batch_size=8,
    collate_fn=data_collator,
)

In [23]:
for batch in train_dataloader:
    break

In [25]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)
outputs = model(**batch)
outputs


SequenceClassifierOutput(
    loss=tensor(0.6969, grad_fn=<NllLossBackward0>),
    logits=tensor([[0.3867, 0.2386],
        [0.3802, 0.2468],
        [0.3638, 0.2169],
        [0.3883, 0.2365],
        [0.4003, 0.2678],
        [0.3653, 0.1906],
        [0.4040, 0.2355],
        [0.3977, 0.2341]], grad_fn=<AddmmBackward0>),
    hidden_states=None,
    attentions=None
)